In [ ]:
!pip install -qU langchain-pinecone pinecone

### Credentials
Create a new Pinecone account, or sign into your existing one, and create an API key to use in this notebook.

In [1]:
import getpass
import os

from pinecone import Pinecone

if not os.getenv("PINECONE_API_KEY"):
    os.environ["PINECONE_API_KEY"] = getpass.getpass("Enter your Pinecone API key: ")

pinecone_api_key = os.environ.get("PINECONE_API_KEY")

pc = Pinecone(api_key=pinecone_api_key)

### Initialization
Before initializing our vector store, let's connect to a Pinecone index. If one named index_name doesn't exist, it will be created.

In [9]:
index_name = "langchain-test-sparse-index"  # change if desired

if not pc.has_index(index_name):
    pc.create_index_for_model(
        name=index_name,
        cloud="aws",
        region="us-east-1",
        embed={
            "model":"pinecone-sparse-english-v0",
            "field_map":{"text": "chunk_text"},
            "metric": "dotproduct",
            "read_parameters": {},
            "write_parameters": {}
        }
    )

index = pc.Index(index_name)
print(f"Index `{index_name}` host: {index.config.host}")

Index `langchain-test-sparse-index` host: https://langchain-test-sparse-index-f62f25f.svc.aped-4627-b74a.pinecone.io


Initialize sparse embeddings

In [11]:
from langchain_pinecone import PineconeSparseEmbeddings

sparse_embeddings = PineconeSparseEmbeddings(model="pinecone-sparse-english-v0")

Now that our Pinecone index is setup, we can initialize our sparse vector store.

In [16]:
from langchain_pinecone import PineconeSparseVectorStore

vector_store = PineconeSparseVectorStore(index=index, embedding=sparse_embeddings)

### Manage vector store
Once you have created your vector store, we can interact with it by adding and deleting different items.

#### Add items to vector store
We can add items to our vector store by using the `add_documents` function.

In [17]:
from uuid import uuid4

from langchain_core.documents import Document

document_1 = Document(
    page_content="I had chocalate chip pancakes and scrambled eggs for breakfast this morning.",
    metadata={"source": "tweet"},
)

document_2 = Document(
    page_content="The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees.",
    metadata={"source": "news"},
)

document_3 = Document(
    page_content="Building an exciting new project with LangChain - come check it out!",
    metadata={"source": "tweet"},
)

document_4 = Document(
    page_content="Robbers broke into the city bank and stole $1 million in cash.",
    metadata={"source": "news"},
)

document_5 = Document(
    page_content="Wow! That was an amazing movie. I can't wait to see it again.",
    metadata={"source": "tweet"},
)

document_6 = Document(
    page_content="Is the new iPhone worth the price? Read this review to find out.",
    metadata={"source": "website"},
)

document_7 = Document(
    page_content="The top 10 soccer players in the world right now.",
    metadata={"source": "website"},
)

document_8 = Document(
    page_content="LangGraph is the best framework for building stateful, agentic applications!",
    metadata={"source": "tweet"},
)

document_9 = Document(
    page_content="The stock market is down 500 points today due to fears of a recession.",
    metadata={"source": "news"},
)

document_10 = Document(
    page_content="I have a bad feeling I am going to get deleted :(",
    metadata={"source": "tweet"},
)

documents = [
    document_1,
    document_2,
    document_3,
    document_4,
    document_5,
    document_6,
    document_7,
    document_8,
    document_9,
    document_10,
]
uuids = [str(uuid4()) for _ in range(len(documents))]

vector_store.add_documents(documents=documents, ids=uuids)

['3f1131d8-ad13-4d5c-9116-69ec4c8e70a0',
 '48fdd7d1-d5c4-4f4e-887d-d09b51d5a7e1',
 'b1b1373e-9854-404b-893c-1c6c559b2133',
 '38587162-422c-41aa-bad9-8dd6c8025d90',
 '4e3c7ee6-a8cb-4a79-9d26-cc3dc6d219eb',
 '43a8646f-dc2e-400e-a213-d0f4af1f4b86',
 'e8cbb839-691f-4a43-a672-fac9cc43cbd2',
 '94e4c135-1787-4fe1-833b-0aca4e8dc736',
 '05033fde-f7f4-4e91-8fe1-dfd8806cbc9f',
 'f69c2039-c194-47e4-9c09-853c3fa788a8']

#### Delete items from vector store

In [18]:
vector_store.delete(ids=[uuids[-1]])

### Query vector store
Once your vector store has been created and the relevant documents have been added you will most likely wish to query it during the running of your chain or agent.

#### Query directly
Performing a simple similarity search can be done as follows:

In [20]:
results = vector_store.similarity_search(
    "LangChain provides abstractions to make working with LLMs easy",
    k=3,
    filter={"source": "tweet"},
)
for res in results:
    print(f"* {res.page_content} [{res.metadata}]")

* Building an exciting new project with LangChain - come check it out! [{'source': 'tweet'}]
* Wow! That was an amazing movie. I can't wait to see it again. [{'source': 'tweet'}]
* I had chocalate chip pancakes and scrambled eggs for breakfast this morning. [{'source': 'tweet'}]


##### Similarity search with score
You can also search with score:

In [21]:
results = vector_store.similarity_search_with_score(
    "Will it be hot tomorrow?", k=1, filter={"source": "news"}
)
for res, score in results:
    print(f"* [SIM={score:3f}] {res.page_content} [{res.metadata}]")

* [SIM=4.640625] The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees. [{'source': 'news'}]


#### Query by turning into retriever
You can also transform the vector store into a retriever for easier usage in your chains.

In [22]:
retriever = vector_store.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={"k": 1, "score_threshold": 0.5},
)
retriever.invoke("Stealing from the bank is a crime", filter={"source": "news"})

/Users/jakit/customers/aurelio/langchain-pinecone/libs/pinecone/.venv/lib/python3.10/site-packages/langchain_core/vectorstores/base.py:1077: UserWarning: Relevance scores must be between 0 and 1, got [(Document(id='38587162-422c-41aa-bad9-8dd6c8025d90', metadata={'source': 'news'}, page_content='Robbers broke into the city bank and stole $1 million in cash.'), 2.3984375)]
  self.vectorstore.similarity_search_with_relevance_scores(


[Document(id='38587162-422c-41aa-bad9-8dd6c8025d90', metadata={'source': 'news'}, page_content='Robbers broke into the city bank and stole $1 million in cash.')]